In [1]:
import tensorflow as tf
import glob
import cv2
import numpy as np
from config import *

def representative_data_gen():
  dataset_list = tf.data.Dataset.list_files(car_images)
  for i in range(100):
    image = next(iter(dataset_list))
    image = tf.io.read_file(image)
    image = tf.io.decode_png(image, channels=3)
    image = tf.image.resize(image, [320, 320])
    image = tf.cast(image / 255., tf.float32)
    image = tf.expand_dims(image, 0)
    yield [image]

converter = tf.lite.TFLiteConverter.from_saved_model(detection_edge_model_tflite)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_data_gen
converter.target_spec.supported_types = [tf.int8]
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.uint8
converter.inference_output_type = tf.uint8
tflite_quant_model = converter.convert()

ConverterError: <unknown>:0: error: loc(callsite(callsite(fused["Postprocessor/BatchMultiClassNonMaxSuppression/MultiClassNonMaxSuppression/ChangeCoordinateFrame/Scale/concat", "Postprocessor/BatchMultiClassNonMaxSuppression/MultiClassNonMaxSuppression/ChangeCoordinateFrame/Scale/concat@__inference_call_func_11386"] at fused["StatefulPartitionedCall", "StatefulPartitionedCall@__inference_signature_wrapper_13760"]) at "StatefulPartitionedCall")): 'tf.ConcatV2' op is neither a custom op nor a flex op
<unknown>:0: note: loc("StatefulPartitionedCall"): called from
<unknown>:0: note: loc(callsite(callsite(fused["Postprocessor/BatchMultiClassNonMaxSuppression/MultiClassNonMaxSuppression/ChangeCoordinateFrame/Scale/concat", "Postprocessor/BatchMultiClassNonMaxSuppression/MultiClassNonMaxSuppression/ChangeCoordinateFrame/Scale/concat@__inference_call_func_11386"] at fused["StatefulPartitionedCall", "StatefulPartitionedCall@__inference_signature_wrapper_13760"]) at "StatefulPartitionedCall")): 

In [ ]:
with open('model.tflite', 'wb') as f:
  f.write(tflite_quant_model)